## Assignment 5
    1. Name: Digambar Chaure
    2. Batch: P-10
    3. Roll No.: 43216
    
   ### Problem Statement :
    Implement the Continuous Bag of Words (CBOW) Model


#### Importing libraries

In [1]:
from keras.preprocessing import text
from tensorflow.keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing import sequence
import numpy as np
import pandas as pd

from keras.utils import to_categorical





#### Taking random sentences as data

In [2]:
data = """Deep learning (also known as deep structured learning) is part of a broader family of machine learning methods based on artificial neural networks with representation learning. Learning can be supervised, semi-supervised or unsupervised. 
Deep-learning architectures such as deep neural networks, deep belief networks, deep reinforcement learning, recurrent neural networks, convolutional neural networks and Transformers have been applied to fields including computer vision, speech recognition, natural language processing, machine translation, bioinformatics, drug design, medical image analysis, climate science, material inspection and board game programs, where they have produced results comparable to and in some cases surpassing human expert performance.
"""
dl_data = data.split()

#### a. Data preparation
#### Tokenization


In [3]:
tokenizer = text.Tokenizer()
tokenizer.fit_on_texts(dl_data)
word2id = tokenizer.word_index

word2id['PAD'] = 0
id2word = {v:k for k, v in word2id.items()}
wids = [[word2id[w] for w in text.text_to_word_sequence(doc)] for doc in dl_data]

vocab_size = len(word2id)
embed_size = 100
window_size = 2 

print('Vocabulary Size:', vocab_size)
print('Vocabulary Sample:', list(word2id.items())[:10])

Vocabulary Size: 75
Vocabulary Sample: [('learning', 1), ('deep', 2), ('networks', 3), ('neural', 4), ('and', 5), ('as', 6), ('of', 7), ('machine', 8), ('supervised', 9), ('have', 10)]


#### b. Generate training data
#### Generating (context word, target/label word) pairs

In [4]:
def generate_context_word_pairs(corpus, window_size, vocab_size):
    context_length = window_size*2
    for words in corpus:
        sentence_length = len(words)
        for index, word in enumerate(words):
            context_words = []
            label_word   = []            
            start = index - window_size
            end = index + window_size + 1
            
            context_words.append([words[i] 
                                 for i in range(start, end) 
                                 if 0 <= i < sentence_length 
                                 and i != index])
            label_word.append(word)

            x = pad_sequences(context_words, maxlen=context_length)
            # y = x;
            y = to_categorical(label_word, vocab_size)
            # y = pad_sequences(label_word, vocab_size)
            yield (x, y)
            
i = 0
for x, y in generate_context_word_pairs(corpus=wids, window_size=window_size, vocab_size=vocab_size):
    if 0 not in x[0]:
        print('Context (X):', [id2word[w] for w in x[0]], '-> Target (Y):', id2word[np.argwhere(y[0])[0][0]])
    
        if i == 10:
            break
        i += 1

#### c. Train model
#### Model building

In [5]:
import keras.backend as K
from keras.models import Sequential
from keras.layers import Dense, Embedding, Lambda

cbow = Sequential()
cbow.add(Embedding(input_dim=vocab_size, output_dim=embed_size, input_length=window_size*2))
cbow.add(Lambda(lambda x: K.mean(x, axis=1), output_shape=(embed_size,)))
cbow.add(Dense(vocab_size, activation='softmax'))
cbow.compile(loss='categorical_crossentropy', optimizer='rmsprop')

print(cbow.summary())

# from IPython.display import SVG
# from keras.utils.vis_utils import model_to_dot

# SVG(model_to_dot(cbow, show_shapes=True, show_layer_names=False, rankdir='TB').create(prog='dot', format='svg'))

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 4, 100)            7500      
                                                                 
 lambda (Lambda)             (None, 100)               0         
                                                                 
 dense (Dense)               (None, 75)                7575      
                                                                 
Total params: 15075 (58.89 KB)
Trainable params: 15075 (58.89 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [6]:
for epoch in range(1, 6):
    loss = 0.
    i = 0
    for x, y in generate_context_word_pairs(corpus=wids, window_size=window_size, vocab_size=vocab_size):
        i += 1
        loss += cbow.train_on_batch(x, y)
        if i % 100000 == 0:
            print('Processed {} (context, word) pairs'.format(i))

    print('Epoch:', epoch, '\tLoss:', loss)
    print()

Epoch: 1 	Loss: 433.63662338256836

Epoch: 2 	Loss: 429.32082533836365

Epoch: 3 	Loss: 426.13176012039185

Epoch: 4 	Loss: 423.06321120262146

Epoch: 5 	Loss: 420.5900616645813



In [7]:
weights = cbow.get_weights()[0]
weights = weights[1:]
print(weights.shape)

pd.DataFrame(weights, index=list(id2word.values())[1:]).head()

(74, 100)


,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
deep,-0.021084,0.006103,-0.031814,0.035423,0.023713,-0.027820,0.031899,-0.048250,-0.031764,-0.049902,...,-0.022906,-0.006347,-0.010172,0.017360,0.026332,0.023315,-0.003002,0.019667,0.031540,-0.001583
networks,-0.000412,-0.042485,-0.030193,0.001367,-0.030932,0.038665,-0.017899,-0.028196,0.026849,0.048810,...,0.023398,0.026378,0.030653,0.043704,-0.042744,-0.047346,-0.015518,0.042783,0.063237,-0.013474
neural,0.008015,0.044186,-0.044066,0.036801,0.038825,0.014629,-0.037670,0.037631,0.013585,-0.000447,...,-0.034125,0.020259,-0.042056,0.016909,-0.006547,-0.016218,0.046122,0.032891,-0.046983,-0.020227
and,0.024419,-0.001714,0.011232,-0.047460,0.014579,-0.022021,-0.045271,0.001647,-0.008147,-0.014263,...,0.007514,0.013822,0.032611,0.048463,0.008977,0.011856,-0.024564,-0.024580,-0.035521,-0.008892
as,-0.046243,0.005224,0.041783,-0.024480,0.046457,0.045561,-0.032513,0.018861,-0.033432,-0.006889,...,-0.023168,-0.014525,0.006973,-0.025727,-0.031680,-0.025969,-0.043915,0.028054,0.028061,0.013622


#### d. Output 

In [8]:
from sklearn.metrics.pairwise import euclidean_distances

distance_matrix = euclidean_distances(weights)
print(distance_matrix.shape)

similar_words = {search_term: [id2word[idx] for idx in distance_matrix[word2id[search_term]-1].argsort()[1:6]+1] 
                   for search_term in ['deep']}

similar_words

(74, 74)


{'deep': ['of', 'such', 'game', 'based', 'with']}